### IMPORTS

In [1]:
import pymc as pm
import numpy as np
import pandas as pd
import arviz as az

pd.get_option("display.max_columns")

WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


20

### READ DATA

In [2]:
anon_prices = pd.read_csv("anonymized_transactions.csv")

### DATA PREP

In [3]:
anon_prices[['transaction_date', 'product_level_1', 'product_level_2', 'zproduct_id']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   transaction_date  999 non-null    object
 1   product_level_1   999 non-null    object
 2   product_level_2   999 non-null    object
 3   zproduct_id       999 non-null    object
dtypes: object(4)
memory usage: 31.3+ KB


In [4]:
anon_prices['transaction_month'] = pd.to_datetime(anon_prices['transaction_date']).dt.month

In [5]:
anon_prices[['transaction_month', 'product_level_1', 'product_level_2', 'zproduct_id']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   transaction_month  999 non-null    int32 
 1   product_level_1    999 non-null    object
 2   product_level_2    999 non-null    object
 3   zproduct_id        999 non-null    object
dtypes: int32(1), object(3)
memory usage: 27.4+ KB


In [6]:
anon_prices[ 'transaction_month'].value_counts()

transaction_month
3     282
1     271
12    231
2     215
Name: count, dtype: int64

In [7]:
anon_prices.describe()

,sm_transaction_0002_key,anchor_weight,test_field_2,targetpricetag,startpricetag,revenue,quantity,profit,price_unit,price_normalizer,...,lee_spend_bin,test_me_bin,snk_spend_bin,cv_split_1,target_nv,start_nv,floor_nv,cv_split_3,cv_split_2,transaction_month
count,0.0,0.0,0.0,0.0,0.0,999.000000,999.000000,999.000000,999.000000,999.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,999.000000
mean,NaN,NaN,NaN,NaN,NaN,2754.158627,244.094285,735.029055,11.547703,8.207517,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.323323
std,NaN,NaN,NaN,NaN,NaN,5691.460171,406.173816,1524.541181,5.555201,3.790929,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.277518
min,NaN,NaN,NaN,NaN,NaN,0.000000,2.000000,-4268.482000,0.000000,1.715260,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
25%,NaN,NaN,NaN,NaN,NaN,607.381150,50.000000,134.671550,7.515714,5.439222,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
50%,NaN,NaN,NaN,NaN,NaN,1369.218900,100.000000,326.965500,10.689700,7.396065,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000
75%,NaN,NaN,NaN,NaN,NaN,2900.379250,200.000000,824.422350,14.766741,10.336887,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000
max,NaN,NaN,NaN,NaN,NaN,76633.801900,4000.000000,19461.298000,69.912030,37.312920,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.000000


In [8]:
anon_prices.head(3)

,sm_transaction_0002_key,transaction_id,anchor_weight,transaction_date,test_field_2,targetpricetag,startpricetag,revenue,region_group,quantity,...,lee_spend_bin,test_me_bin,snk_spend_bin,cv_split_1,target_nv,start_nv,floor_nv,cv_split_3,cv_split_2,transaction_month
0,NaN,2-039995466:25,NaN,2019-12-03 00:00:00.000000,NaN,NaN,NaN,2924.7128,B9,300.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12
1,NaN,2-039997979:21,NaN,2019-12-05 00:00:00.000000,NaN,NaN,NaN,26427.2451,SK,3000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12
2,NaN,2-040000029:11,NaN,2019-12-09 00:00:00.000000,NaN,NaN,NaN,4188.5119,WLS,500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12


### Data Prep: Bayesian Hierarchical Regression

In [9]:
df_params = {"hbr_b_mu": -0.25,
             "hbr_b_sig": 0.05,
             "Test_Time_Index": [12,13],
             "Min_Elast": 0.5,
             "Max_Elast": 2.5}

hbr_b_mu = float(df_params['hbr_b_mu'])
hbr_b_sig = float(df_params['hbr_b_sig'])

anon_prices['MCMC_Partition'] = 'Global_Partition'

MCMC_Partition = np.unique(anon_prices['MCMC_Partition'])

bayes_hier = anon_prices[['MCMC_Partition',
                          'product_level_1', 
                          'product_level_2', 
                          'zproduct_id', 
                          'transaction_month', 
                          'quantity', 
                          'revenue']]

bayes_tree = bayes_hier[['product_level_1',
                         'product_level_2',
                         'zproduct_id']].columns
tree_levels = len(bayes_tree)

### Metrics: Bayesian Hierarchical Regression: Statistics

In [10]:
groupby_list = ['product_level_1', 'product_level_2', 'zproduct_id', 'transaction_month']
quantity_list = ['product_level_1', 'product_level_2', 'zproduct_id', 'transaction_month', 'quantity']
revenue_list = ['product_level_1', 'product_level_2', 'zproduct_id', 'transaction_month', 'revenue']

bayes_quantity = bayes_hier[quantity_list].groupby(by=groupby_list).sum()
bayes_revenue = bayes_hier[revenue_list].groupby(by=groupby_list).sum()
bayes_avg_price = bayes_revenue['revenue'] / bayes_quantity['quantity']

bayesXmean = bayes_avg_price.mean()
bayesXstd = bayes_avg_price.std()

bayesYmean = bayes_quantity['quantity'].mean()
bayesYstd = bayes_quantity['quantity'].std()

bayesX_zscore =  (bayes_avg_price - bayesXmean) / bayesXstd
bayesY_zscore =  (bayes_quantity - bayesYmean) / bayesYstd

bayesX_zscore = bayesX_zscore.to_numpy()
bayesY_zscore = bayesY_zscore['quantity'].to_numpy()

### Metrics: Bayesian Hierarchical Regression: Levels

In [11]:
new_bayes = bayes_hier[['MCMC_Partition', 'product_level_1', 'product_level_2', 'zproduct_id']]
sort_by_list1 = ['product_level_1']
sort_by_list2 = ['product_level_1', 'product_level_2']
sort_by_list3 = ['product_level_1', 'product_level_2', 'zproduct_id']

In [11]:
# level 1
new_bayes['level1']= new_bayes.sort_values(by=sort_by_list1)\
                               .groupby(by=['MCMC_Partition'])\
                               .cumcount() + 1

# level 2
new_bayes['level2'] = new_bayes.sort_values(by=sort_by_list2)\
                               .groupby(by=['MCMC_Partition'])\
                               .cumcount() + 1

# level 3
new_bayes['level3'] = new_bayes.sort_values(by=sort_by_list3)\
                               .groupby(by=['MCMC_Partition'])\
                               .cumcount() + 1

/tmp/ipykernel_1736820/2646393165.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_bayes['level1']= new_bayes.sort_values(by=sort_by_list1)\
/tmp/ipykernel_1736820/2646393165.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_bayes['level2'] = new_bayes.sort_values(by=sort_by_list2)\


In [15]:
level_1 = new_bayes['level1'].values
level_1_unique = np.unique(level_1)
level_1_count = len(level_1_unique)

level_2 = new_bayes['level2'].values
level_2_link = new_bayes[['level1', 'level2']].copy().drop_duplicates()['level1'].values
level_2_unique = np.unique(level_2)
level_2_count = len(level_2_unique)

level_3 = new_bayes['level3'].values
level_3_link = new_bayes[['level2', 'level3']].copy().drop_duplicates()['level2'].values
level_3_unique = np.unique(level_3)
level_3_count = len(level_3_unique)

### PROGRAM

In [21]:
with pm.Model() as hierarchical_model:

    b_0 = pm.Normal(name='b_0', mu=hbr_b_mu, sigma=hbr_b_sig, shape=1)

    b_1 = pm.Normal(name='b_1', mu=b_0,  sigma=hbr_b_sig, shape=level_1_count)

    b_2 = pm.Normal(name='b_2', mu=b_1[level_2_link - 1],  sigma=hbr_b_sig, shape=level_2_count)        

    b_3 = pm.Normal(name='b_3', mu=b_2[level_3_link - 1],  sigma=hbr_b_sig, shape=level_3_count)

    eps = pm.HalfCauchy(name='eps', beta=1)

    # Likelihood
    Y_est = b_3[level_3 - 1] * bayesX_zscore
    # Y_like = pm.Normal(name='Y_like', mu=Y_est, sigma=eps, observed=bayesY_zscore)        
    
    trace = pm.sample(draws=2000, tune=2000, cores=4, chains=2, random_seed=1, progressbar=0)

b_est = trace['b_3'].mean(axis=0)
results = pd.DataFrame({'Level_Key': level_3_unique, 'MCMC_Partition': 'Global_Partition', 'b_est': b_est})


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [b_0, b_1, b_2, b_3, eps]


ValueError: Not enough samples to build a trace.